In [ ]:
from pathlib import Path

from rich import print

In [111]:
test_files = list(Path.cwd().parent.glob('data/**/*.pdf'))

In [112]:
test_files

[PosixPath('/home/thankgod/2025/tiiqu/textract-tiquu/data/JSR_1778.pdf'),
 PosixPath('/home/thankgod/2025/tiiqu/textract-tiquu/data/JSR_1774.pdf'),
 PosixPath('/home/thankgod/2025/tiiqu/textract-tiquu/data/What_is_Sustainability-1.pdf'),
 PosixPath('/home/thankgod/2025/tiiqu/textract-tiquu/data/JSR_1775.pdf')]

In [3]:
# from marker.converters.pdf import PdfConverter
# from marker.models import create_model_dict
# from marker.output import text_from_rendered

# converter = PdfConverter(
#     artifact_dict=create_model_dict(),
# )
# rendered = converter("/home/thankgod/2025/tiiqu/textract-tiquu/data/What_is_Sustainability-1.pdf")
# text, _, images = text_from_rendered(rendered)

In [91]:
# from marker.converters.pdf import PdfConverter
# from marker.models import create_model_dict
# from marker.config.parser import ConfigParser

# config = {
#     "output_format": "chunks",
#     "ADDITIONAL_KEY": "VALUE"
# }
# config_parser = ConfigParser(config)

# converter = PdfConverter(
#     config=config_parser.generate_config_dict(),
#     artifact_dict=create_model_dict(),
#     processor_list=config_parser.get_processors(),
#     renderer=config_parser.get_renderer(),
#     llm_service=config_parser.get_llm_service()
# )
# rendered1 = converter(filepath=file_path1)
# rendered2 = converter(filepath=file_path2)

In [29]:
#print(rendered)

In [ ]:
from marker.config.parser import ConfigParser
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict

config = {
    "output_format": "markdown",
    "ADDITIONAL_KEY": "VALUE"
}
config_parser = ConfigParser(config)

converter = PdfConverter(
    config=config_parser.generate_config_dict(),
    artifact_dict=create_model_dict(),
    processor_list=config_parser.get_processors(),
    renderer=config_parser.get_renderer(),
    llm_service=config_parser.get_llm_service()
)

rendered_files = [converter(filepath=str(test_file)) for test_file in test_files]

Recognizing Text: 100%|██████████| 1/1 [00:00<00:00,  2.02it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Running OCR Error Detection: 100%|██████████| 2/2 [00:00<00:00, 23.51it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Running OCR Error Detection: 100%|██████████| 1/1 [00:00<00:00, 20.94it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Running OCR Error Detection: 100%|██████████| 2/2 [00:00<00:00, 24.69it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing tables: 100%|██████████| 1/1 [00:00<00:00,  3.08it/s]


In [116]:
#print(rendered_files[0].markdown)

In [87]:
pdf_text = rendered.markdown
#print(pdf_text)

In [ ]:
import re


def extract_headers(markdown_text):
    # Pattern to match headers 1-3 (# ## ###)
    pattern = r'^(#{1,10})\s+(.+)$'
    
    headers = []
    for line in markdown_text.split('\n'):
        match = re.match(pattern, line.strip())
        if match:
            level = len(match.group(1))  # Number of # symbols
            title = match.group(2).strip()
            headers.append({'level': level, 'title': title})
    
    return headers

In [121]:
headers = extract_headers(rendered_files[3].markdown)
for header in headers:
    print(f"Level {header['level']}: {header['title']}")

Level 4: *Article*

Level 1: **Assessing the University of Johannesburg's Engagement and Position in Global Sustainable Development 
Goal Practices**

Level 4: **Chidubem Deborah Adamu, Kathija Yassim \*, Chinaza Uleanya \***

Level 4: **ABSTRACT**

Level 4: **Open Access**

...

Level 2: **DATA AVAILABILITY**

Level 4: **AUTHOR CONTRIBUTIONS**

Level 2: **CONFLICTS OF INTEREST**

Level 4: **ACKNOWLEDGMENTS**

Level 3: **REFERENCES**

In [122]:
def extract_headers(markdown_text):
    """Extract headers 1-10 from markdown text"""
    pattern = r'^(#{1,10})\s+(.+)$'
    headers = []
    for line in markdown_text.split('\n'):
        match = re.match(pattern, line.strip())
        if match:
            level = len(match.group(1))  # Number of # symbols
            title = match.group(2).strip()
            headers.append({'level': level, 'title': title})
    return headers

In [184]:
def is_valid_header(header_text):
   """
   Check if a header is valid based on simple heuristics.
   
   Args:
       header_text: The header text to validate
   
   Returns:
       bool: True if header is valid, False otherwise
   """
   if not header_text or not header_text.strip():
       return False
   
   # Remove markdown formatting for analysis
   clean_text = header_text.replace('*', '').replace('#', '').strip()
   
   # Quick length checks
   if len(clean_text) == 0:
       return False
   if len(clean_text) > 200:
       return False
   
   # Invalid patterns - simple checks
   invalid_patterns = [
       r'<[^>]*>',  # HTML/XML tags
       r'[\\\/]',   # Contains backslash or forward slash
       r"[']",      # Contains single quotes
   ]
   
   # Check against invalid patterns
   for pattern in invalid_patterns:
       if re.search(pattern, clean_text, re.IGNORECASE):
           return False
   
   return True

In [185]:
def extract_between_headers(text, start_header, end_header, validate_headers=True):
    """
    Extract text between two headers with optional header validation.
    
    Args:
        text: Full text content
        start_header: Starting header (e.g., "2. History of the Concept")
        end_header: Ending header (e.g., "3. People, Planet, Profit")
        validate_headers: If True, validate headers before extraction (default: True)
    
    Returns:
        Extracted text between headers, or None if headers not found or invalid
    """
    #Validate headers if requested
    if validate_headers:
        if not is_valid_header(start_header):
            #print(f"Warning: Invalid start header detected: '{start_header}'")
            return None
        
        if not is_valid_header(end_header):
            #print(f"Warning: Invalid end header detected: '{end_header}'")
            return None
    
    # Find start position
    start_pos = text.find(start_header)
    if start_pos == -1:
        return None
    
    # Move past the start header
    start_pos += len(start_header)
    
    # Find end position
    end_pos = text.find(end_header, start_pos)
    if end_pos == -1:
        return None
    
    # Extract and clean the text
    extracted = text[start_pos:end_pos].strip()
    return extracted

In [186]:
def count_words(text):
    """Count words in text"""
    return len(text.split())

def find_valid_header_pairs(text, min_words=250):
    """
    Find pairs of consecutive headers where text between them is at least min_words.
    
    Args:
        text: Full text content
        min_words: Minimum number of words between headers (default: 250)
    
    Returns:
        List of tuples: [(start_header, end_header, extracted_text), ...]
    """
    headers = extract_headers(text)
    valid_pairs = []
    
    # Check each consecutive pair of headers
    for i in range(len(headers) - 1):
        start_header = headers[i]['title']
        end_header = headers[i + 1]['title']
        
        # Extract text between current pair
        extracted_text = extract_between_headers(text, start_header, end_header)
        
        if extracted_text and count_words(extracted_text) >= min_words:
            valid_pairs.append((start_header, end_header, extracted_text))
    
    return valid_pairs

In [187]:
def process_text_sections(text, min_words=250):
    """
    Simple function to get valid text sections between headers.
    
    Args:
        text: Full text content
        min_length: Minimum length of text between headers (default: 250)
    
    Returns:
        List of extracted text sections that meet the length requirement
    """
    valid_pairs = find_valid_header_pairs(text, min_words)
    return [section[2] for section in valid_pairs]  # Return just the text content


In [188]:
rendered_files[0].markdown

'### *Article*\n\n# **Urban Development and Land Use/Land Cover Changes in the European Borderlands of Nice and Monaco**\n\n### **Arian Behradfar 1,2, Rui Alexandre Castanho 2,3,\\*, Paulo Ferreira <sup>2</sup>**\n\n- <sup>1</sup> Department of Geomatics and Spatial Information Engineering, College of Engineering, University of Tehran, Tehran 14174-66191, Iran; arianbehradfar@ut.ac.ir (AB)\n- <sup>2</sup> VALORIZA—Research Centre for Endogenous Resource Valorization, Portalegre Polytechnic University, Portalegre 7300-110, Portugal; pferreira@ipportalgre.pt (PF)\n- <sup>3</sup> Advanced Research Centre, European University of Lefke, Lefke 99010, Northern Cyprus, TR-10 Mersin, Turkey\n- **\\*** Correspondence: Rui Alexandre Castanho, Email: acastanho@wsb.edu.pl, rui.castanho@ipportalegre.pt, Tel.: +351-912-494-673\n\n# **ABSTRACT**\n\nUrban development in European cross-border regions significantly influences LULC changes, affecting environmental sustainability and regional planning. Thi

In [189]:

sample_text = rendered_files[2].markdown

# Find valid header pairs
valid_pairs = find_valid_header_pairs(sample_text, min_words=250)
    
print("Valid header pairs (with text >= 250 words):")
for i, (start, end, content) in enumerate(valid_pairs, 1):
    word_count = count_words(content)
    print(f"\n{i}. From '{start}' to '{end}'")
    print(f"   Word count: {word_count} words")
    print(f"   Preview: {content[:100]}...")

Valid header pairs (with text >= 250 words):

1. From '**2. History of the Concept**' to '**3. People, Planet, Profit**'

Word count: 528 words

Preview: The concept of sustainability was originally coined in forestry, where it means never harvesting mor...

2. From '**4. Happiness, Well-being and Welfare**' to '**5. Sustainability: Resources and the Future**'

...

Word count: 809 words

Preview: Sustainability, then, is a matter of what resources—natural resources, quality of the environment, a...

5. From '**7. Conclusions**' to '**References and Notes**'

Word count: 894 words

Preview: Having reached this point, the reader may feel we have come no closer to solving the practical probl...

In [172]:
#print(vars(rendered))